In [9]:
# Imports
%load_ext autoreload
%autoreload 2
%matplotlib inline
f
import csv
import importlib
from scripts import proj1_helpers, helpers
from scripts import implementation, feature_processing, k_fold, model_linear
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
output_path = '../data/logreg_submission.csv'

In [ ]:
# loading data
y, X, idx = proj1_helpers.load_csv_data(train_path)
y_t, X_t, ids_t = proj1_helpers.load_csv_data(test_path)

In [ ]:
X_p = feature_processing.process_X(X)
X_t_p = feature_processing.process_X(X_t)

In [13]:
X_p

array([[  1.00000000e+00,   3.14910656e-01,   6.83319669e-02, ...,
         -6.70555834e-01,   1.99057407e+00,  -3.11898221e-01],
       [  1.00000000e+00,   7.40827026e-01,   5.52504823e-01, ...,
          1.49130013e+00,  -5.02367642e-01,  -3.11898221e-01],
       [  1.00000000e+00,  -9.69035940e-13,   3.19515553e+00, ...,
          1.49130013e+00,  -5.02367642e-01,  -3.11898221e-01],
       ..., 
       [  1.00000000e+00,  -3.10930673e-01,   3.19316447e-01, ...,
          1.49130013e+00,  -5.02367642e-01,  -3.11898221e-01],
       [  1.00000000e+00,  -5.10097335e-01,  -8.45323970e-01, ...,
         -6.70555834e-01,  -5.02367642e-01,  -3.11898221e-01],
       [  1.00000000e+00,  -9.69035940e-13,   6.65336083e-01, ...,
         -6.70555834e-01,  -5.02367642e-01,  -3.11898221e-01]])

In [15]:
model = implementation.ridge_regression
loss = model_linear.compute_loss_reg

idx_min, rmse_all, lambdas = k_fold.cross_validation_select(X_p, y, model, loss, seed = 1, k_fold = 5, N = 20, do_plot = True)

KeyboardInterrupt: 

## Bias-Variance Decomposition
Visualize bias-variance trade-off by implementing the function `bias_variance_demo()` below:

In [ ]:
from implementation import least_squares
from split_data import split_data
from plots import bias_variance_decomposition_visualization

def bias_variance_demo(ridge_lambda = 0, use_cv = 0):
    """The entry."""
    # define parameters
    seeds = range(100)
    num_data = 5000
    ratio_train = 0.005
    degrees = range(1, 10)
    
    k_fold = 4
    
    # define list to store the variable
    rmse_tr = np.zeros((len(seeds), len(degrees)))
    rmse_te = np.zeros((len(seeds), len(degrees)))
    
    for index_seed, seed in tqdm(enumerate(seeds)):
        np.random.seed(seed)
        x = np.linspace(0.1, 2 * np.pi, num_data)
        y = np.sin(x) + 0.3 * np.random.randn(num_data).T
        
        idx_tr, idx_te = split_data(x, y, ratio_train, seed = seed)
        
        for index_degree, degree in enumerate(degrees):
            # building polynomial matrix for whole dataset
            #x_poly = build_poly(x, degree)
            y,tx = build_model_data(x,y)

        
            # x data: train/test
            x_tr = tx[idx_tr, :]
            x_te = tx[idx_te, :]
    
            # y data: train/test
            y_tr = y[idx_tr]
            y_te = y[idx_te]

            if use_cv:
                # selecting lambda:
                idx_min, rmse_all, lambdas = cross_validation_select(np.array(x)[idx_tr], y_tr, seed, degree, k_fold)
                ridge_lambda = lambdas[idx_min]
            
            # training ridge regression
            if ridge_lambda > 0:
                weights = ridge_regression(y_tr, x_tr, ridge_lambda)
            else:
                weights = least_squares(y_tr, x_tr)

            # computing losses
            rmse_tr[index_seed, index_degree] = compute_mse(y_tr, x_tr, weights)
            rmse_te[index_seed, index_degree] = compute_mse(y_te, x_te, weights)

    plt.figure()
    bias_variance_decomposition_visualization(degrees, rmse_tr, rmse_te)
    plt.boxplot(rmse_tr, positions = degrees)
    plt.boxplot(rmse_te, positions = degrees)
    plt.show()

In [ ]:
# least squares
bias_variance_demo()

In [ ]:
# ridge
bias_variance_demo(ridge_lambda = 1)

In [ ]:
# ridge with CV for lambda
bias_variance_demo(use_cv = 1)

In [ ]:
from proj1_helpers import predict_labels
from proj1_helpers import create_csv_submission

def build_model(lambda_val,name):
    X_t = impute_with_mean(x_test)
    X_ts, _, _ = helpers.standardize(X_t)
    y_t, tx_t = helpers.build_model_data(X_ts, y_test)
    
    y_m,tx = build_model_data(x,y)
    weights = ridge_regression(y_m, tx, lambda_val)
    
    y_pred=predict_labels(weights,tx_t)
    create_csv_submission(ids_test,y_pred,name)


In [ ]:
build_model(0.000001,"submit_csv")